# Constructing OLAP cubes

### Introduction

In this lesson, we'll practice using SQL queries to construct our OLAP cubes.  We'll work up towards construct the query for the movie rental database that we've used previously. Let's get started.

### Getting Set Up

Now we have previously used the movie database in an OLTP structure, but in this lesson, we'll need to also create the tables for a star schema, and copy over our data into that structure.

* To do this, uncomment and run the command below to create the `pagila_starred` database in postgres.

In [12]:
# !PGPASSWORD=postgres createdb -h 127.0.0.1 -U postgres pagila_starred

* And then uncomment and run the command below to create the tables and load in the corresponding data.

In [ ]:
# !PGPASSWORD=postgres psql -q -h 127.0.0.1 -U postgres -d pagila_starred -f data/pagila_star.sql

Then we'll declare the following function to connect to the database.

In [1]:
import psycopg2

def get_cursor():
    conn = psycopg2.connect(
    host="127.0.0.1",
    database="pagila_starred",
    user="postgres",
    password="postgres")
    cursor = conn.cursor()
    return cursor

Ok, next create the cursor.

In [2]:
cursor = get_cursor()

And then we can begin to explore the database.  

So log into postgres, connect to the database, and get a sense of the different tables.

> <img src="./pagila_relations.png" width="40%">

If you look at the database, we can see that while we have multiple tables in the database, only the ones beginning with `dim` or `fact` are meant for our OLAP.

Now take a look at the columns in the fact table.

> <img src="./factsales.png" width="80%">

And let's take a look at one of the dimension tables, say the `dimmovie` columns.

> <img src="./dimmovie.png" width="80%">

> **Note**:  We can see that the dimension table has a primary key of `movie_key`, instead of our typical `id` column.

Overall, our star schema takes on the same form that we have seen previously. 

<img src="./star_schemad_movies.png" width="60%">

With this structure in mind, let's begin to make our queries.

### Making the queries

So instead diving right into the query for an OLAP cube, let's instead build up to it.

1. Begin by finding the number of sales made in each movie rating category -- order from highest to lower by number of sales.

In [5]:
query = """
SELECT dimmovie.rating, COUNT(factsales.sales_key) as total_sales FROM factsales 
JOIN dimmovie ON factsales.movie_key = dimmovie.movie_key GROUP BY dimmovie.rating
ORDER BY total_sales DESC LIMIT 10;
"""

In [6]:
cursor.execute(query)

In [7]:
cursor.fetchall()
# [('PG-13', 3585), ('NC-17', 3298), ('PG', 3212), ('R', 3181), ('G', 2773)]

[('PG-13', 3585), ('NC-17', 3298), ('PG', 3212), ('R', 3181), ('G', 2773)]

So we can see that the most popular movies were PG-13, and the least popular were rated G.

2. Next, group the data by store number and rating -- showing the number of sales made across both dimensions.

> So this would be the equivalent of the following diagram (the answers will differ).

> <img src="./two_dim_group.png" width="30%">

To accomplish this, we will need to group by multiple columns.  If unsure how to group by multiple columns, see the [following post](https://stackoverflow.com/questions/2421388/using-group-by-on-multiple-columns). 

In [ ]:
query = """
SELECT dimmovie.rating, COUNT(factsales.sales_key) as total_sales FROM factsales 
JOIN dimmovie ON factsales.movie_key = dimmovie.movie_key GROUP BY (dimmovie.rating, dimmovie.rating) 
ORDER BY total_sales DESC LIMIT 10;
"""
cursor.execute(query)
agg_sales = cursor.fetchall()
agg_sales


# [('PG-13', 1, 1849),
#  ('PG-13', 2, 1736),
#  ('PG', 2, 1677),
#  ('NC-17', 2, 1668),
#  ('R', 2, 1644),
#  ('NC-17', 1, 1630),
#  ('R', 1, 1537),
#  ('PG', 1, 1535),
#  ('G', 2, 1396),
#  ('G', 1, 1377)]

So above, we can see that our sales are divided across two dimensions: the rating and the store.  And we show the total sales broken out for each.  

> Now we can use some pandas to make it easier to view the results.  It's not important that you understand the below queries at this point.  We'll go through this later.

In [118]:
import pandas as pd
sales_df = pd.DataFrame(agg_sales)
sales_df.pivot_table(values=2, 
                     index=sales_df[0],
                     columns=sales_df[1],
                     aggfunc='first')

1,1,2
0,,
G,1377,1396
NC-17,1630,1668
PG,1535,1677
PG-13,1849,1736
R,1537,1644


So as we can see, this is turning into a nice little dashboard.  Sales by store are along the horizontal axis, and sales by rating are along the vertical axis.  So currently we have the "what" on the y axis, and the "where" along the x axis.  Let's add in the "when" along the z axis.  To do so, we'll add to our previous query by grouping by year.

In [127]:
query = """
SELECT dimdate.month, dimmovie.rating, factsales.store_key, COUNT(factsales.sales_key) as total_sales FROM factsales 
JOIN dimmovie ON factsales.movie_key = dimmovie.movie_key 
JOIN dimdate ON factsales.date_key = dimdate.date_key 
GROUP BY (dimmovie.rating, factsales.store_key, dimdate.month)
ORDER BY total_sales DESC LIMIT 10;
"""

In [128]:
cursor.execute(query)

In [129]:
cubed_results = cursor.fetchall()

In [130]:
cubed_results

# [(4, 'PG-13', 1, 797),
#  (4, 'PG-13', 2, 727),
#  (4, 'PG', 2, 704),
#  (4, 'NC-17', 2, 690),
#  (4, 'R', 2, 679),
#  (4, 'NC-17', 1, 666),
#  (4, 'PG', 1, 651),
#  (4, 'R', 1, 643),
#  (3, 'PG-13', 1, 635),
#  (3, 'NC-17', 2, 609)]

[(4, 'PG-13', 1, 797),
 (4, 'PG-13', 2, 727),
 (4, 'PG', 2, 704),
 (4, 'NC-17', 2, 690),
 (4, 'R', 2, 679),
 (4, 'NC-17', 1, 666),
 (4, 'PG', 1, 651),
 (4, 'R', 1, 643),
 (3, 'PG-13', 1, 635),
 (3, 'NC-17', 2, 609)]

One of the things we can notice is that the only sales that exists for March are PG-13 and NC-17 movies.  So this may explain at least part of the disproportionate sales of those movies  -- it looks like the rest of the sales data is missing.  Either way by breaking this data out across three dimensions, we were able to better spot some anomalies in our data.

### Summary

In this lesson, we saw how to construct the queries for our OLAP cubes.  We did so by joining together the required tables, and then by grouping our data across various dimensions.  As we saw, doing so allowed us to view aggregated information across multiple dimensions.

### Resources

[Crosstab function](https://learnsql.com/blog/creating-pivot-tables-in-postgresql-using-the-crosstab-function/)